In [2]:
import tkinter as tk
import numpy as np
import matplotlib
from scipy.stats import poisson
import seaborn as sns
from scipy.stats import kurtosis
from scipy.stats import skew
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

In [3]:
def detector_simulator(lraw_counts, ang_solido, laceitancia_geom, leficiencia_detector):
    total_solido = round((lraw_counts*laceitancia_geom*leficiencia_detector), 2)
    return total_solido

In [4]:
def fonte_de_radiacao(lmi, lq_medidas, lj, ltempo_medida):
    data_poisson = poisson.rvs(mu = lmi, size = lq_medidas)
    #num_aleatorio = random.choice(data_poisson)
    return(data_poisson[int(lj)]*ltempo_medida)

In [5]:
def angulo_solido(lR, lh):
    global ang_solido
    r = np.sqrt(lh**2 + lR**2)
    sen_teta1 = lR/r
    teta = np.arcsin(sen_teta1)
    ang_solido = round(2*np.pi*(1-np.cos(teta)),2)
    print('Ângulo sólido =', angulo_solido, 'rad')
    return ang_solido

In [6]:
h = 0
j = 0
janela_aberta = False
janela_aberta2 = False
newWindow = 0
newWindowGraph = 0
ativ = 0
num_medidas = 0
tempo_medida = 0
voltagem = 0
ang_solido = 0
list_total_medida = []
list_total_solido = []
#Função para movimentar o detector
def arrastarCursor(event):
    if dentro(event.x, event.y) and event.y > 50 and event.y < 400:
        global h
        canvas.coords(detector, posicao_1[0], event.y-25, posicao_2[0], event.y+25)
        c = canvas.coords(detector)
        h = 434-c[3]
#Função que detecta se o mouse esta dentro da área do detector
def dentro(x, y):
    c = canvas.coords(detector)
    #print(c)
    if x > c[0] and x < c[2] and y < c[3] and y > c[1]:
        return True
#Função que cria o formato da fonte
def create_circle(x, y, r, canvas): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    return canvas.create_oval(x0, y0, x1, y1, fill = 'black')
#Função para fechar a segunda janela
def fechaJanela():
    global janela_aberta
    global newWindow
    janela_aberta = False
    newWindow.destroy()
#Função para criar as caixas de texto 
def inputText():
    global ativ
    global num_medidas
    global tempo_medida
    global voltagem
    ativ = float(ativtxt.get(1.0, 'end-1c'))
    num_medidas = int(num_medidastxt.get(1.0, 'end-1c'))
    tempo_medida = float(tempo_medidatxt.get(1.0, 'end-1c'))
    voltagem = float(voltagemtxt.get(1.0, 'end-1c'))
#Função do botão Pronto
def teste():
    np.savetxt("numpy_test.csv", list_total_solido, delimiter =",",fmt ='% s')
    
def clickGraph():
    global janela_aberta2
    global newWindowGraph
    global list_total_solido
    if not janela_aberta2:
        #Definindo a nova janela
        #newWindowGraph = tk.Toplevel(root)
        #newWindowGraph.title("Histograma")
        #newWindowGraph.geometry("500x500")
        #Plot
        sns.displot(list_total_solido)
        #newWindowGraph.protocol('WM_DELETE_WINDOW', fechaJanelaGraph)
def clickPronto():
    global janela_aberta
    global newWindow
    global j 
    global ativ
    global num_medidas
    global tempo_medida
    global voltagem
    global list_total_solido
    global list_total_medida
    if not janela_aberta:
        #Definindo a nova janela
        newWindow = tk.Toplevel(root)
        newWindow.title("Resultados")
        newWindow.geometry("500x500")
        inputText()
        #Rodasr funções externas
        list_total_solido = []
        list_total_medida =[]
        ang_solido = angulo_solido(125, h)
        aceitancia_geom = ang_solido/(4*np.pi)
        eficiencia_detector = 0.01*((2/25)*voltagem+24)/100
        while j < num_medidas:
            raw_counts = fonte_de_radiacao(ativ, num_medidas, j, tempo_medida)
            counts = detector_simulator(raw_counts, ang_solido, aceitancia_geom, eficiencia_detector)
            list_total_medida.append(raw_counts)
            list_total_solido.append(counts)
            j+=1
        media_total = round(sum(list_total_medida)/len(list_total_medida), 2)
        media_solido = sum(list_total_solido)/len(list_total_solido)
        desvio_padrao_total = round(np.std(list_total_medida), 2)
        desvio_padrao_solido = np.std(list_total_solido)
        #skewness_total = skew(list_total_medida)
        #skewness_medida = skew(list_total_solido)
        #kurtosis_total = kurtosis(list_total_medida)
        #kurtosis_medida = kurtosis(list_total_solido)
        print('Média de decaimentos =', media_total, '\nMédia de decaimentos detectados =', media_solido)
        print('Desvio padrão total =', desvio_padrao_total, '\nDesvio padrão medido =', desvio_padrao_solido)
        #print('Skewness total =', skewness_total, '\nSkewness das medidas =', skewness_medida)
        #print('Kurtosis total =', kurtosis_total, '\nKurtosis das medidas =', kurtosis_medida) 
        tk.Label(newWindow, text = "Distância: " + str(h)).pack()
        tk.Label(newWindow, text = "Ângulo sólido: " + str(ang_solido)).pack()
        #tk.Label(newWindow, text = "Atividade da fonte: " + str(ativ)).pack()
        tk.Label(newWindow, text = "Média: " + str(media_total)).pack()
        tk.Label(newWindow, text = "Desvio padrão: " + str(desvio_padrao_total)).pack()
        newWindow.protocol('WM_DELETE_WINDOW', fechaJanela)
        j = 0
        janela_aberta = True

    else:
        fechaJanela()
        clickPronto()

root = tk.Tk()
root.configure(bg = 'grey')
root.geometry("500x500")
root.title('Detector')

#Objetos
Botao = tk.Button(root, text = "Pronto!", fg = "black", font = 32, command = clickPronto)
Botao2 = tk.Button(root, text = "Histograma", fg = 'black', font = 32, command = clickGraph)
txt1 = tk.Label(root, text = "Insirsa a atividade da fonte em Curie!", bg = 'grey', font = 28)
txt2 = tk.Label(root, text = "Insira o número de medidas desejadas!", bg = 'grey', font = 28)
txt3 = tk.Label(root, text = "Insirsa o tempo de cada medida em seugundos!", bg = 'grey', font = 28)
txt4 = tk.Label(root, text = "Insirsa a voltagem no detector!", bg = 'grey', font = 28)

#Criar desenhos
canvas = tk.Canvas(root, bg = 'grey', width = 550, height = 820)

posicao_1 = [50, 375]
posicao_2 = [300, 425]

detector = canvas.create_rectangle(posicao_1[0], posicao_1[1], posicao_2[0], posicao_2[1], fill='red')
fonte = create_circle(175, 435, 10, canvas)
ativtxt = tk.Text(root, height = 2, width = 20)
num_medidastxt = tk.Text(root, height = 2, width = 20)
tempo_medidatxt = tk.Text(root, height = 2, width = 20)
voltagemtxt = tk.Text(root, height = 2, width = 20)

canvas.bind('<B1-Motion>', arrastarCursor)
#Barra de Menu
barraDeMenus = tk.Menu(root)
menuOpcoes = tk.Menu(barraDeMenus, tearoff = 0)
menuOpcoes.add_command(label='Salvar', command=teste)
barraDeMenus.add_cascade(label='Arquivo', menu = menuOpcoes)
root.config(menu=barraDeMenus)
#Posição de desenhos
canvas.place(x = 200, y = 0)
#Posição dos botões
Botao.place(x = 1350, y = 700, height = 50, width = 150)
Botao2.place(x = 1150, y = 700, height = 50, width = 150)
#Posição dos textos
txt1.place(x = 1100, y = 50)
txt2.place(x = 1100, y = 150)
txt3.place(x = 1100, y = 250)
txt4.place(x = 1100, y = 350)
#Posição das caixas
ativtxt.place(x = 1100, y = 100)
num_medidastxt.place(x = 1100, y = 200)
tempo_medidatxt.place(x = 1100, y = 300)
voltagemtxt.place(x = 1100, y = 400)

root.mainloop()